In [ ]:
import folium
import pyproj
import momepy
import shapely
import osmnx as ox
import pandas as pd
import networkx as nx
import geopandas as gpd
import matplotlib.pyplot as plt

from tqdm import tqdm
from shapely.wkt import loads, dumps
from shapely.ops import nearest_points
from shapely.geometry import Point, Polygon, LineString, MultiLineString

# from dongraphio import DonGraphio, GraphType
# from dongraphio.base_models import BuildsGrapher

In [ ]:
for u, v, key, d in G.edges(data=True, keys=True):
    if 'geometry' in d:
        d['geometry'] = loads(d['geometry'])

In [ ]:
test_roads = gpd.read_file('test_roads.geojson')

In [ ]:
n, e = ox.graph_to_gdfs(G, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)

In [ ]:
e.to_file('e.geojson', driver='GeoJSON')

In [ ]:
import geopandas as gpd
from shapely.geometry import MultiLineString

def unpack_multilinestrings_to_linestrings(gdf):
    # Создаем пустой список для хранения данных нового GeoDataFrame
    rows_list = []
    # Итерируем по каждой строке исходного GeoDataFrame
    for index, row in gdf.iterrows():
        geometry = row.geometry
        # Проверяем, является ли геометрия MultiLineString
        if isinstance(geometry, MultiLineString):
            # Если да, то итерируем по каждому LineString, используя .geoms
            for linestring in geometry.geoms:
                # Создаем новую строку с теми же атрибутами, но с одним LineString
                new_row = row.copy()
                new_row.geometry = linestring
                rows_list.append(new_row)
        else:
            # Если геометрия уже является LineString, просто добавляем строку в список
            rows_list.append(row)
    # Создаем новый GeoDataFrame из списка строк
    new_gdf = gpd.GeoDataFrame(rows_list, columns=gdf.columns)
    # Устанавливаем ту же систему координат, что и у исходного gdf
    new_gdf.crs = gdf.crs
    return new_gdf

# Пример использования функции:
# gdf - это ваш исходный GeoDataFrame с MultiLineStrings
# new_gdf - это новый GeoDataFrame с LineStrings
# new_gdf = unpack_multilinestrings_to_linestrings(df)

In [ ]:
G = nx.read_graphml('car_graph.graphml')
G

In [ ]:
roads = gpd.read_file('roads.geojson')

In [ ]:
df = roads[roads['YEAR'].notna()]

In [ ]:
new_gdf = unpack_multilinestrings_to_linestrings(df)

In [ ]:
new_gdf = new_gdf.to_crs(epsg=32638)

In [ ]:
new_gdf.columns

In [ ]:
G_future = momepy.gdf_to_nx(new_gdf,directed=True)

In [ ]:
G_future, G

In [ ]:
for u, v, d in G_future.edges(data=True):
    d['type'] = 'car'

G_future.graph['crs'] = 'epsg:32638'
for node in G_future.nodes:
    x = node[0]
    y = node[1]
    G_future.nodes()[node]['x'] = x
    G_future.nodes()[node]['y'] = y

In [ ]:
mapping = {old_id: new_id for new_id, old_id in enumerate(G.nodes())}
G = nx.relabel_nodes(G, mapping)

mapping = {old_id: new_id+1000000 for new_id, old_id in enumerate(G_future.nodes())}
G_future = nx.relabel_nodes(G_future, mapping)

In [ ]:
for u, v, key, d in G_future.edges(data=True, keys=True):
    if 'geometry' in d:
        d['geometry'] = dumps(d['geometry'])

In [ ]:
G.graph, list(G.nodes(data=True))[0], list(G.edges(data=True))[0], G_future.graph, list(G_future.nodes(data=True))[0], list(G_future.edges(data=True))[0]

In [ ]:
G_copy = G.copy()
G_future_copy = G_future.copy()

In [ ]:
BG = BuildsGrapher(city_crs=32638, city_osm_id=176095)

In [ ]:
BG.graphs_spatial_union(G_copy, G_future_copy)

In [ ]:
def download_graph(id, simplify=True):
    gdf = ox.geocode_to_gdf(id, by_osmid=True)
    polygon_boundary = gdf.unary_union
    graph = ox.graph_from_polygon(polygon_boundary,
                                  network_type='drive',
                                  simplify=simplify)
    G = nx.Graph(graph)
    return G
    
G = download_graph('R176095')

In [ ]:
spb = ox.geocode_to_gdf('R337422', by_osmid=True) # Санкт-Петербург
lo = ox.geocode_to_gdf('R176095', by_osmid=True)  # Ленинградская область

spb_buff = spb.to_crs(epsg=3857).buffer(1000).to_crs(epsg=4326)
lo_buff = lo.to_crs(epsg=3857).buffer(3000).to_crs(epsg=4326)

In [ ]:
lo_filter = {'highway': ['motorway', 'trunk', 'primary', 'secondary', 
                           'tertiary', 'unclassified', 'residential', 
                           'motorway_link', 'trunk_link', 'primary_link', 'secondary_link', 'tertiary_link']}

spb_filter = {'highway': ['motorway', 'trunk', 'primary', 
                           'motorway_link', 'trunk_link', 'primary_link']}

polygon_boundary = result_polygon.unary_union
polygon_boundary_spb = spb_buff.unary_union


lo_gdf = ox.features_from_polygon(polygon_boundary,
                                 tags=lo_filter)

spb_gdf = ox.features_from_polygon(polygon_boundary_spb,
                                 tags=spb_filter)

lo_gdf = lo_gdf[lo_gdf['geometry'].geom_type == 'LineString']
# spb_gdf = spb_gdf[spb_gdf['highway'].isin(['motorway', 'trunk', 'primary', 'motorway_link', 'trunk_link', 'primary_link'])]

In [ ]:
roads_within_polygon = gpd.sjoin(lo_gdf, lo, how='inner', predicate='intersects')
roads_within_polygon.reset_index(inplace=True)
lo_gdf.shape, roads_within_polygon.shape

In [ ]:
polygon = lo.union(spb_buff)
gdf_polygon = gpd.GeoDataFrame(geometry=polygon)

In [ ]:
# Создание нового атрибута со значением 0 для всех дорог
roads_within_polygon['new_attribute'] = 0
test = roads_within_polygon.intersects(gdf_polygon.unary_union.boundary)
roads_within_polygon.loc[roads_within_polygon[test].index.values, 'new_attribute'] = 1
graph = momepy.gdf_to_nx(roads_within_polygon.to_crs(epsg=4326))

In [ ]:
graph = momepy.gdf_to_nx(roads_within_polygon.to_crs(epsg=4326))

In [ ]:
dongrph = DonGraphio(city_crs=32638)
    
intermodal_graph = dongrph.get_intermodal_graph_from_osm(city_osm_id=176095,keep_city_boundary=False)
nx.write_graphml(intermodal_graph,"city_intermodal.graphml")

builds_from = gpd.read_file("data/Границы+население/admin_centers_LO_188_points.geojson")
services_to = gpd.read_file("data/Границы+население/admin_centers_LO_188_points.geojson")
adjacency_matrix = dongrph.get_adjacency_matrix(gdf_from=builds_from, gdf_to=services_to, weight="time_min", graph_type=[GraphType.DRIVE])
adjacency_matrix.to_csv("city_adjacency_matrix.csv")

In [ ]:
G = nx.read_graphml('city_intermodal.graphml')

In [ ]:
list(G.edges(data=True))[0]

In [ ]:
unique_types = set(edge[2]['type'] for edge in G.edges(data=True))
print(unique_types)

In [ ]:
from dongraphio.utils import get_subgraph
get_subgraph(G, "type", [t.value for t in sum([t.edges for t in [GraphType.PUBLIC_TRANSPORT,GraphType.WALK], [])],)


In [ ]:
G_nx = G.edge_subgraph([(u, v, k) for u, v, k, d in G.edges(data=True, keys=True) if d['type'] in ['car']])

In [ ]:
list(G_nx.edges(data=True))[0]

In [ ]:
list(G_nx.nodes(data=True))[0]

In [ ]:
merged_gdf.to_file('roads_OSM.geojson', driver='GeoJSON')

In [ ]:
roads = gpd.read_file('roads.geojson')

In [ ]:
import folium

# Создаем карту
m = folium.Map(tiles='CartoDB Dark_Matter')

folium.GeoJson(
    merged_gdf,
    name='OSM',
    style_function=lambda feature: {
        'fillColor': '#ffaf00',  # Цвет заливки
        'color': 'blue',         # Цвет границы
        'weight': 1.5,           # Толщина границы
    }
).add_to(m)

folium.GeoJson(
    roads[roads['YEAR'].notna()],
    name='Natasha',
    style_function=lambda feature: {
        'fillColor': '#0080ff',  # Цвет заливки
        'color': 'yellow',          # Цвет границы
        'weight': 1.5,           # Толщина границы
    }
).add_to(m)

folium.LayerControl().add_to(m)

m